In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [2]:
daily = pd.read_csv('../data/call-center-data-v2-daily.csv')
hoo = pd.read_csv('../data/call-center-data-v2-daily-hoo.csv')

In [3]:
daily

,Date,Incoming Calls,Answered Calls,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG)
0,2022-01-01,157,145,12,0:00:15,0:02:29,0:03:12
1,2022-01-02,37,37,0,0:00:03,0:02:06,0:00:35
2,2022-01-03,317,304,13,0:00:18,0:01:35,0:02:37
3,2022-01-04,253,244,9,0:00:13,0:01:50,0:02:02
4,2022-01-05,214,205,9,0:00:10,0:02:10,0:03:22
...,...,...,...,...,...,...,...
1242,2025-05-27,203,195,8,0:00:11,0:02:47,0:01:52
1243,2025-05-28,192,184,8,0:00:07,0:02:50,0:01:56
1244,2025-05-29,212,209,3,0:00:10,0:02:51,0:01:45
1245,2025-05-30,211,203,8,0:00:12,0:03:22,0:03:52


In [4]:
hoo

,Date,Incoming Calls,Answered Calls,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG)
0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,317.0,304.0,13.0,0:00:18,0:01:35,0:02:37
3,2022-01-04,253.0,244.0,9.0,0:00:13,0:01:50,0:02:02
4,2022-01-05,214.0,205.0,9.0,0:00:10,0:02:10,0:03:22
...,...,...,...,...,...,...,...
1242,2025-05-27,203.0,195.0,8.0,0:00:11,0:02:47,0:01:52
1243,2025-05-28,192.0,184.0,8.0,0:00:07,0:02:50,0:01:56
1244,2025-05-29,212.0,209.0,3.0,0:00:10,0:02:51,0:01:45
1245,2025-05-30,211.0,203.0,8.0,0:00:12,0:03:22,0:03:52


## Data Cleaning

In [5]:
# Convert 'Date' to datetime
daily['Date'] = pd.to_datetime(daily['Date'])
hoo['Date'] = pd.to_datetime(hoo['Date'])

# Convert time-string columns to total seconds
for col in ['Answer Speed (AVG)', 'Talk Duration (AVG)', 'Waiting Time (AVG)']:
    daily[col] = pd.to_timedelta(daily[col]).dt.total_seconds()
    hoo[col] = pd.to_timedelta(hoo[col]).dt.total_seconds()

# Extract the day of week: Monday=0, Sunday=6
daily['Day Of Week'] = daily['Date'].dt.dayofweek
hoo['Day Of Week'] = hoo['Date'].dt.dayofweek

In [6]:
# Identify missing values (NaN) in the hoo dataset's 'Incoming Calls'.
# .isna() returns True/False. .astype(int) converts True to 1, False to 0.
hoo['Is Non Operational'] = hoo['Incoming Calls'].isna().astype(int)

# Merge the 'Is_Non_Operational' column from 'hoo' into 'daily' using the Date index.
daily = daily.merge(
    # Select only the two necessary columns from hoo: the key and the new flag
    hoo[['Date', 'Is Non Operational']], 
    on='Date',
    how='left'
)

In [7]:
# Calculate Abandonment Rate
daily['Abandonment Rate (%)'] = np.where(
    daily['Incoming Calls'] > 0,
    (daily['Abandoned Calls'] / daily['Incoming Calls']) * 100,
    0
)

In [8]:
# Define top 25% and bottom 25% thresholds for Incoming Calls
q75 = daily['Incoming Calls'].quantile(0.75)
q25 = daily['Incoming Calls'].quantile(0.25)

daily['Volume Segment'] = 'Medium'
daily.loc[daily['Incoming Calls'] >= q75, 'Volume Segment'] = 'High'
daily.loc[daily['Incoming Calls'] <= q25, 'Volume Segment'] = 'Low'

In [9]:
daily = daily.sort_values('Date')
daily = daily.set_index('Date')

In [10]:
daily

,Incoming Calls,Answered Calls,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG),Day Of Week,Is Non Operational,Abandonment Rate (%),Volume Segment
Date,,,,,,,,,,
2022-01-01,157,145,12,15.0,149.0,192.0,5,1,7.643312,Medium
2022-01-02,37,37,0,3.0,126.0,35.0,6,1,0.000000,Low
2022-01-03,317,304,13,18.0,95.0,157.0,0,0,4.100946,High
2022-01-04,253,244,9,13.0,110.0,122.0,1,0,3.557312,High
2022-01-05,214,205,9,10.0,130.0,202.0,2,0,4.205607,Medium
...,...,...,...,...,...,...,...,...,...,...
2025-05-27,203,195,8,11.0,167.0,112.0,1,0,3.940887,Medium
2025-05-28,192,184,8,7.0,170.0,116.0,2,0,4.166667,Medium
2025-05-29,212,209,3,10.0,171.0,105.0,3,0,1.415094,Medium


## Modeling

In [11]:
# Data manipulation
import pandas as pd
import numpy as np

# Time series / stats models
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pmdarima as pm
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Machine learning models
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Model evaluation
from sklearn.metrics import mean_absolute_error

# Optional / utility
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# train/test split (chronological)
train = daily[daily.index < '2025-03-01']
test = daily[daily.index >= '2025-03-01']

print("train range:", train.index.min(), "->", train.index.max(), "n=", len(train))
print("test range:", test.index.min(),  "->", test.index.max(),  "n=", len(test))

train range: 2022-01-01 00:00:00 -> 2025-02-28 00:00:00 n= 1155
test range: 2025-03-01 00:00:00 -> 2025-05-31 00:00:00 n= 92


In [13]:
train

,Incoming Calls,Answered Calls,Abandoned Calls,Answer Speed (AVG),Talk Duration (AVG),Waiting Time (AVG),Day Of Week,Is Non Operational,Abandonment Rate (%),Volume Segment
Date,,,,,,,,,,
2022-01-01,157,145,12,15.0,149.0,192.0,5,1,7.643312,Medium
2022-01-02,37,37,0,3.0,126.0,35.0,6,1,0.000000,Low
2022-01-03,317,304,13,18.0,95.0,157.0,0,0,4.100946,High
2022-01-04,253,244,9,13.0,110.0,122.0,1,0,3.557312,High
2022-01-05,214,205,9,10.0,130.0,202.0,2,0,4.205607,Medium
...,...,...,...,...,...,...,...,...,...,...
2025-02-24,393,363,30,19.0,186.0,174.0,0,0,7.633588,High
2025-02-25,283,280,3,6.0,185.0,109.0,1,0,1.060071,High
2025-02-26,241,238,3,5.0,175.0,196.0,2,0,1.244813,High


## Naive baseline: Forecast using prior year demand
MAE: 300.446

MASE: 1.000

WMAPE: 168.092

In [34]:
def forecast_metrics(y_true, y_pred, y_train, season_length=365):

    mae = np.mean(np.abs(y_true - y_pred))

    # Seasonal naive forecast: use prior year's value
    if len(y_train) > season_length:
        naive_forecast = y_train[:-season_length]
        actual_future = y_train[season_length:]
        mase_denom = np.mean(np.abs(actual_future - naive_forecast))
    else:
        mase_denom = np.nan  # Not enough history to compute properly

    mase = mae / mase_denom if mase_denom != 0 else np.nan
    wmape = 100 * np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

    return {"MAE": mae, "MASE": mase, "WMAPE (%)": wmape}

## Model 1: Simple Moving Average
Predict today = average of previous 7 days (rolling(7).mean()) or just use the value 7 days ago (lag_7)

In [38]:
# 1. Define forecast horizon
test_dates = test.index

# 2. Initialize list to store predictions
sma_forecast = []

# 3. Walk-forward forecast
history = train['Incoming Calls'].copy()

for current_date in test_dates:
    # Compute the 7-day simple moving average
    forecast = history[-7:].mean()
    sma_forecast.append(forecast)
    
    # Update history with the actual test value
    history.loc[current_date] = test.loc[current_date, 'Incoming Calls']

# 4. Convert predictions to a Series aligned with test set
sma_forecast = pd.Series(sma_forecast, index=test_dates)

# 5. Evaluate metrics using helper
sma_metrics = forecast_metrics(
    test['Incoming Calls'].values, 
    sma_forecast.values, 
    train['Incoming Calls'].values
)

sma_mae, sma_mase, sma_wmape = (
    sma_metrics['MAE'], 
    sma_metrics['MASE'], 
    sma_metrics['WMAPE (%)']
)

print(f"SMA 7-day Forecast Metrics:\nMAE: {sma_mae:.2f}\nMASE: {sma_mase:.3f}\nWMAPE: {sma_wmape:.2f}%")

SMA 7-day Forecast Metrics:
MAE: 76.67
MASE: 0.516
WMAPE: 42.89%


## Model 2: SARIMA

In [40]:
# 1. Check stationarity
result = adfuller(train['Incoming Calls'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -4.917338760935737
p-value: 3.226315822581374e-05


In [43]:
# 2. Fit auto_arima on the train set
sarima_model = pm.auto_arima(
    train['Incoming Calls'],
    seasonal=True,
    m=7,                      # Weekly seasonality
    stepwise=True,
    trace=True,
    error_action='ignore',
    suppress_warnings=True
)

print(sarima_model.summary())

# 3. Forecast for the test period
n_periods = len(test)
sarima_forecast = sarima_model.predict(n_periods=n_periods)
sarima_forecast = pd.Series(sarima_forecast, index=test.index)

# 4. Evaluate metrics using helper function
sarima_mae, sarima_mase, sarima_wmape = forecast_metrics(
    train['Incoming Calls'],
    test['Incoming Calls'],
    sarima_forecast
)

sarima_metrics = forecast_metrics(
    y_true=test['Incoming Calls'].values,
    y_pred=sarima_forecast.values,
    y_train=train['Incoming Calls'].values,
    season_length=365
)

sarima_mae = sarima_metrics["MAE"]
sarima_mase = sarima_metrics["MASE"]
sarima_wmape = sarima_metrics["WMAPE (%)"]

print(f"SARIMA Forecast Metrics:")
print(f"MAE: {sarima_mae:.2f}")
print(f"MASE: {sarima_mase:.3f}")
print(f"WMAPE: {sarima_wmape:.2f}%")

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[7] intercept   : AIC=inf, Time=1.53 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=14815.719, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=14366.380, Time=0.24 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=14387.010, Time=0.24 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=14813.720, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[7] intercept   : AIC=14648.937, Time=0.03 sec
 ARIMA(1,1,0)(2,0,0)[7] intercept   : AIC=14275.114, Time=0.44 sec
 ARIMA(1,1,0)(2,0,1)[7] intercept   : AIC=inf, Time=1.85 sec
 ARIMA(1,1,0)(1,0,1)[7] intercept   : AIC=14084.206, Time=0.82 sec
 ARIMA(1,1,0)(0,0,1)[7] intercept   : AIC=14474.861, Time=0.24 sec
 ARIMA(1,1,0)(1,0,2)[7] intercept   : AIC=inf, Time=1.81 sec
 ARIMA(1,1,0)(0,0,2)[7] intercept   : AIC=14385.461, Time=0.43 sec
 ARIMA(1,1,0)(2,0,2)[7] intercept   : AIC=14088.180, Time=2.35 sec
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=14202.074, Time=0.57 sec
 ARIMA(2,1,0)(1,0,1)[7] intercept   :

## Model 3: Random Forest Regressor

In [45]:
df = train.copy()

# Lag features (previous 7 days)
for lag in range(1, 8):
    df[f'lag_{lag}'] = df['Incoming Calls'].shift(lag)

# Rolling mean
df['rolling_7'] = df['Incoming Calls'].shift(1).rolling(7).mean()

# Day of week and weekend indicator
df['day_of_week'] = df.index.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)

# Drop first 7 days with NaN
df = df.dropna()

# Prepare test set
test_df = test.copy()
full = pd.concat([train, test])

for lag in range(1, 8):
    test_df[f'lag_{lag}'] = full['Incoming Calls'].shift(lag).loc[test.index]

test_df['rolling_7'] = full['Incoming Calls'].shift(1).rolling(7).mean().loc[test.index]
test_df['day_of_week'] = test_df.index.dayofweek
test_df['is_weekend'] = test_df['day_of_week'].isin([5,6]).astype(int)
test_df = test_df.dropna()

# Features and target
features = [f'lag_{i}' for i in range(1,8)] + ['rolling_7','day_of_week','is_weekend']
X_train = df[features]
y_train = df['Incoming Calls']
X_test = test_df[features]
y_test = test_df['Incoming Calls']

# Train model
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Forecast
rf_forecast = rf_model.predict(X_test)

# Evaluate
rf_metrics = forecast_metrics(y_true=y_test.values, y_pred=rf_forecast, y_train=y_train.values)

rf_mae = rf_metrics["MAE"]
rf_mase = rf_metrics["MASE"]
rf_wmape = rf_metrics["WMAPE (%)"]

print("Random Forest Forecast Metrics:")
print(f"MAE: {rf_mae:.2f}")
print(f"MASE: {rf_mase:.3f}")
print(f"WMAPE: {rf_wmape:.2f}%")

Random Forest Forecast Metrics:
MAE: 52.87
MASE: 0.355
WMAPE: 29.58%


## Model 4: LightGBM

In [46]:
lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=5,
    random_state=42
)

# Train model
lgb_model.fit(X_train, y_train)

# Forecast
lgb_forecast = lgb_model.predict(X_test)

# Evaluate
lgb_metrics = forecast_metrics(y_true=y_test.values, y_pred=lgb_forecast, y_train=y_train.values)

lgb_mae = lgb_metrics["MAE"]
lgb_mase = lgb_metrics["MASE"]
lgb_wmape = lgb_metrics["WMAPE (%)"]

print("LightGBM Forecast Metrics:")
print(f"MAE: {lgb_mae:.2f}")
print(f"MASE: {lgb_mase:.3f}")
print(f"WMAPE: {lgb_wmape:.2f}%")

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2049
[LightGBM] [Info] Number of data points in the train set: 1148, number of used features: 10
[LightGBM] [In

## Model 5: XG Boost

In [47]:
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Train model
xgb_model.fit(X_train, y_train)

# Forecast
xgb_forecast = xgb_model.predict(X_test)

# Evaluate
xgb_metrics = forecast_metrics(y_true=y_test.values, y_pred=xgb_forecast, y_train=y_train.values)

xgb_mae = xgb_metrics["MAE"]
xgb_mase = xgb_metrics["MASE"]
xgb_wmape = xgb_metrics["WMAPE (%)"]

print("XGBoost Forecast Metrics:")
print(f"MAE: {xgb_mae:.2f}")
print(f"MASE: {xgb_mase:.3f}")
print(f"WMAPE: {xgb_wmape:.2f}%")

XGBoost Forecast Metrics:
MAE: 52.63
MASE: 0.354
WMAPE: 29.44%


## Summary

In [48]:
# Create a summary dictionary
summary_dict = {
    "Model": ["SMA (7-day)", "SARIMA", "Random Forest", "LightGBM", "XGBoost"],
    "MAE": [sma_mae, sarima_mae, rf_mae, lgb_mae, xgb_mae],
    "MASE": [sma_mase, sarima_mase, rf_mase, lgb_mase, xgb_mase],
    "WMAPE (%)": [sma_wmape, sarima_wmape, rf_wmape, lgb_wmape, xgb_wmape]
}

# Convert to DataFrame
summary_df = pd.DataFrame(summary_dict)

# Optional: sort by MAE (best performing first)
summary_df = summary_df.sort_values("MAE").reset_index(drop=True)

# Display
print("Forecast Model Comparison Summary:")
display(summary_df)

Forecast Model Comparison Summary:


,Model,MAE,MASE,WMAPE (%)
0,XGBoost,52.629344,0.353660,29.444780
1,Random Forest,52.874837,0.355309,29.582127
2,LightGBM,54.215284,0.364317,30.332073
3,SMA (7-day),76.669255,0.516371,42.894499
4,SARIMA,102.599768,0.691014,57.401962
